In [ ]:
import pandas as pd
import numpy as np

read json file and create dataframe

In [ ]:

json_file = 'at_postData_secondbatch.json'
df = pd.read_json(json_file, lines=True)

function for getting number of days 

In [ ]:
def td_days(td):
    return td.days

In [ ]:
#group by id and experimental status
id_group = df.groupby(["id","isExperimental"])
#create list for post dfs
df_list = []
#loop thorugh unqiue posts
for (post_id, isexperimental), group_df in id_group:
    #get time of first post observation
    first_time = group_df['datetime'].min()
    #calculate time since first observation for all observation
    time_passed = group_df['datetime'] - first_time
    #calculate number of days passed since first observation
    days_passed = np.array(time_passed.apply(td_days))
    #get posts which have been observed at 7 days since first observation
    if 7 in days_passed:
        group_df['days_passed'] = days_passed
        #remove potential duplicate observations due to bugs
        group_df = group_df.drop_duplicates(subset=['id', 'days_passed'], keep = 'first')
        #get observations form day one to seven
        group_df = group_df[group_df['days_passed'] <= 7][1:]
        #append post df to list 
        df_list.append(group_df)


Writing the data to a file

In [ ]:
#create dataframe
all_df = pd.concat(df_list)
#convert datframe to .csv file
all_df.to_csv('second_batch_wdays.csv', sep = ';', index = False)